# Federated Protein Embeddings and Task Model Fitting with BioNeMo

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This notebook will walk you through the task fitting workflow in the following sections:

* 
*
*

### Install requirements
Let's start by installing and importing library dependencies. We'll use requests to interact with the BioNeMo service, BioPython to parse FASTA sequences into SeqRecord objects, scikit-learn for classification tasks, and matplotlib for visualization.

In [ ]:
#!pip install -r requirements.txt
#!pip install -e /home/hroth/Code/nvflare/bionemo_nvflare
#!pip install biopython scikit-learn matplotlib

### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, users can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [4]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [5]:
import io
import requests
from Bio import SeqIO

# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

Downloaded 13949 sequences


### Data splitting
Next, we prepare the data for simulating federated learning using `n_clients`.

In [6]:
n_clients = 1
# limiting to the proteins with sequence length<512 for embedding queries
MAX_SEQUENCE_LEN = 512
seed=0
out_dir = "/tmp/fasta/mixed_soft"
split_alpha = 100.0  # moderate label heterogeneity of alpha=1.0

import os
import re
import numpy as np
import pandas as pd
import uuid

from importlib import reload
import split_data
reload(split_data)
from split_data import split, list_to_dataframe
np.random.seed(seed)

# Extract meta data and split
data = []
for i, x in enumerate(proteins):
        if len(str(x.seq)) > MAX_SEQUENCE_LEN:
            continue
            
        entry = {key: value for key, value in re.findall(r"([A-Z_]+)=(-?[A-z0-9]+[.0-9]*)", x.description)}
        entry["sequence"] = str(x.seq)
        entry["id"] = str(i)
       
        data.append(entry)
print(f"Read {len(data)} valid sequences.")
               
# Split the data and save for each client
# Note, test_data is kept the same on each client and is not split
split(proteins=data, num_sites=n_clients, split_dir=out_dir, alpha=split_alpha)  # use `concat=True` if using separate inference + MLP classifier

Read 8619 valid sequences.
Partition protein dataset with 10 classes into 1 sites with Dirichlet sampling under alpha 100.0
{'site-1': {'Cell_membrane': 514,
            'Cytoplasm': 1163,
            'Endoplasmic_reticulum': 462,
            'Extracellular': 1332,
            'Golgi_apparatus': 164,
            'Lysosome': 149,
            'Mitochondrion': 938,
            'Nucleus': 1696,
            'Peroxisome': 89,
            'Plastid': 412}}
Saved 6918 training and 1700 testing proteins for site-1, (10/10) unique train/test classes.


### Federated embedding extraction
Running inference of the ESM-1nv model to extract embeddings requires a GPU with at least 12 GB memory. Here we run inference on each client sequentially using one thread to preserve GPU memory.

In [ ]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/embeddings",
    workspace="/tmp/nvflare/bionemo/embeddings",
    n_clients=n_clients,
    threads=1  # due to memory constraints, we run the client execution sequentially in one thread
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo server are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM-1nv, `D=768`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [ ]:
# load embeddings from site-1
import pickle
protein_embeddings = pickle.load(open(os.path.join(out_dir, "data_site-1.pkl"), "rb"))
print(f"Loaded {len(protein_embeddings)} embeddings from site-1.")

for i in range(4):
    protein_embedding = protein_embeddings[i]
    print(f"Inference result contains {list(protein_embedding.keys())}")
    x = protein_embedding["embeddings"]
    print(f"{protein_embedding['id']}: range {np.min(x)}-{np.max(x)}, mean={np.mean(x)}, shape={x.shape}")

Let's enumerate the labels corresponding to potential subcellular locations.

In [ ]:
# Let's also print all the labels

labels = set([entry['TARGET'] for entry in data])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier. The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the call below, we define the MLP to use the Adam optimizer with a network of 32 hidden layers, defining a random state (or seed) for reproducibility, and trained for a maximum of 500 iterations.

### Local training

In [ ]:
os.environ["SIM_LOCAL"] = "True"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace=f"/tmp/nvflare/bionemo/local_alpha{split_alpha}",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

### Federated learning

In [ ]:
import os
os.environ["SIM_LOCAL"] = "False"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace=f"/tmp/nvflare/bionemo/fedavg_alpha{split_alpha}",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

## Finetuning ESM1nv
#### Federated Learning

In [ ]:
# data preprocessing
#!python jobs/fedavg_finetune_esm1nv/app/custom/downstream_flip.py do_training=False

In [ ]:
# DEBUG
import os
os.environ["SIM_LOCAL"] = "False"
from nvflare import SimulatorRunner    
n_clients = 1
split_alpha = 100.0

simulator = SimulatorRunner(
    #job_folder="jobs/fedavg_finetune_esm1nv",
    #workspace=f"/tmp/nvflare/bionemo/fedavg_finetune_esm1nv_alpha{split_alpha}",
    #workspace=f"/tmp/nvflare/bionemo/local_site1_finetune_esm1nv_alpha{split_alpha}_unfreeze_encoder4",
    job_folder="jobs/fedavg_finetune_esm2nv",
    #workspace=f"/tmp/nvflare/bionemo/local_site1_finetune_esm2nv_alpha{split_alpha}_freeze_encoder2",
    workspace=f"/tmp/nvflare/bionemo/local_site1_finetune_esm2nv_alpha{split_alpha}_unfreeze_encoder1_large_ds",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-12-06 18:40:40,753 - SimulatorRunner - INFO - Create the Simulator Server.
2023-12-06 18:40:40,758 - CoreCell - INFO - server: creating listener on tcp://0:38571
2023-12-06 18:40:40,781 - CoreCell - INFO - server: created backbone external listener for tcp://0:38571
2023-12-06 18:40:40,782 - ConnectorManager - INFO - 261821: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-12-06 18:40:40,784 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:11179] is starting
2023-12-06 18:40:41,287 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:11179
2023-12-06 18:40:41,288 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:38571] is starting
2023-12-06 18:40:41,384 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 39331
2023-12-06 18:40:41,385 - SimulatorRunner - INFO - Deploy the Apps.
2023-12-06 18:40:41,392 - SimulatorRunner - INFO - Creat